In [1]:
import langchain
import getpass
import os


In [2]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key:  ········


## Abordagem de Roteamento de Query usando Function Calling

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.tools import tool
import pydantic
import pandas as pd
import time

In [4]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=1,
    max_tokens=1000,
    max_retries=2,
)

In [5]:
@tool
def get_movie_recommendation(year: int, genre: str) -> str:
    """Recomenda um filme específico com base no ano e no gênero"""
    return f"Titanic, um filme de {genre} de {year}."

@tool
def query_movie_database(name: str) -> str:
    """Consulta informações sobre um filme específico"""
    return f"O filme {name} tem uma ótima avaliação no IMDb."

@tool
def ensinar_cachorro(name: str) -> str:
    """Ensina um cachorro a ser mais rápido e inteligente"""
    return f"Ensinando o cachorro {name} a pular."

@tool
def fazer_cafe(quantidade: str) -> str:
    """Prepara café em uma quantidade específica"""
    return f"Fazendo {quantidade} de café."

In [6]:
tools = [get_movie_recommendation, query_movie_database, ensinar_cachorro, fazer_cafe]

In [7]:
llm_with_tools = llm.bind_tools(tools)

In [23]:
perguntas = [
    "Me recomenda um filme de ação de 2010?",
    "Quais filmes de ficção científica você recomendaria?",
    "Qual o nome do filme que foi lançado em 2005?",
    "Me fale sobre o filme O Senhor dos Anéis",
    "Como faço para ensinar meu cachorro Spark a ser mais rápido?",
    "Me recomenda um filme de terror?",
    "Você sabe fazer bolacha?",
    "Como recomenda fazer pão?",
    "Me faça 600ml de café",
    "Me fale sobre o filme Titanic",
    "Quero todos os cafés",
    "Me ajude a trocar o pneu",
    "Oi"
]


In [27]:
def generate_answers(perguntas):
    respostas = []
    
    # Para cada pergunta, cria uma mensagem e invoca o modelo
    for pergunta in perguntas:
        time.sleep(1)
        message = HumanMessage(content=pergunta)

        ai_msg = llm_with_tools.invoke([message])
        
        # respostas.append((pergunta, ai_msg))
    
        yield (pergunta, ai_msg)

In [28]:
result = []

In [29]:
for pergunta, resposta in generate_answers(perguntas):
    if len(resposta.tool_calls) > 0:
        for tool_call in resposta.tool_calls:
            selected_tool = {
                    "get_movie_recommendation": get_movie_recommendation, 
                    "query_movie_database": query_movie_database,
                    "ensinar_cachorro": ensinar_cachorro,
                    "fazer_cafe": fazer_cafe
            }[tool_call["name"].lower()]
            # print(selected_tool)
            tool_output = selected_tool.invoke(tool_call["args"])
            result.append(
                {
                    "query_input": str(pergunta),
                    "tool_id": str(tool_call["id"]),
                    "args": str(tool_call["args"]),
                    "name": str(tool_call["name"]),
                    "tool_output": str(tool_output)
                }
            )
    else:
         result.append(
                {
                    "query_input": str(pergunta),
                    "tool_id": None,
                    "args": None,
                    "name": None,
                    "tool_output": None
                }
            )

In [30]:
df = pd.DataFrame(result)

In [31]:
df

,query_input,tool_id,args,name,tool_output
0,Me recomenda um filme de ação de 2010?,0fec9af2-f623-40e0-a2f0-6bdb97a30185,"{'year': 2010.0, 'genre': 'ação'}",get_movie_recommendation,"Titanic, um filme de ação de 2010."
1,Quais filmes de ficção científica você recomen...,None,None,None,None
2,Qual o nome do filme que foi lançado em 2005?,None,None,None,None
3,Me fale sobre o filme O Senhor dos Anéis,None,None,None,None
4,Como faço para ensinar meu cachorro Spark a se...,255773c6-da4c-4d7a-8b47-c7653d9f183d,{'name': 'Spark'},ensinar_cachorro,Ensinando o cachorro Spark a pular.
5,Me recomenda um filme de terror?,None,None,None,None
6,Você sabe fazer bolacha?,None,None,None,None
7,Como recomenda fazer pão?,None,None,None,None
8,Me faça 600ml de café,483c1614-d346-4184-aeb5-0f6e3e0d80cd,{'quantidade': '600ml'},fazer_cafe,Fazendo 600ml de café.
9,Me fale sobre o filme Titanic,72ddba7d-645b-4444-8e3d-d6181680830e,{'name': 'Titanic'},query_movie_database,O filme Titanic tem uma ótima avaliação no IMDb.
